In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
# ESC to plot
# also defines plotting order (first to last = upper left to lower right)
esc_order = [
    "hvdc-to-h2",
    "pipeline-h2",
    "shipping-lh2",
    "shipping-lohc",
    "pipeline-ch4",
    "pipeline-ch4-to-h2",
    "shipping-lch4",
    "shipping-lch4-to-h2",
    "shipping-lnh3",
    "shipping-lnh3-to-h2",
    "shipping-meoh",
    "shipping-meoh-to-h2",
    "shipping-ftfuel",
]

# exporter colors and prettier ESC names from config
exp_colors = snakemake.config["plotting"]["exporter_colors"]
esc_pretty_names = snakemake.config["plotting"]["esc_pretty_names"]
esc_markers = snakemake.config["plotting"]["esc_markers"]

# Select data of interest
df = pd.read_csv(snakemake.input[0], sep=";")

df = df.loc[
    (df["scenario"] == snakemake.wildcards["scenario"])
    & (df["esc"].isin(esc_order))
    & (df["year"] == int(snakemake.wildcards["year"]))
    & (df["subcategory"].str.startswith("Total produced electricity from"))
]

# Nicer column entries
df["subcategory"] = df["subcategory"].str.replace("Total produced electricity from","")

# Change format of DataFrame for stacked plotting
df = df.reset_index().pivot(
    index=["esc", "exporter"], columns=["subcategory"], values="value"
)

df.columns.name = ""

number_of_subplot_columns = 4
fig, axes = plt.subplots(number_of_subplot_columns,
                         int(np.ceil(len(esc_order)/number_of_subplot_columns)),
                         figsize=(10, 10),
                         #sharex=True,
                         sharey=True,
                        )
axes_iter = iter(axes.flatten())

plot_legend=True
for esc in esc_order:
    ax = next(axes_iter)
    df.loc[esc].plot(
        kind="bar",
        stacked=True,
        ax=ax,
        title=esc_pretty_names[esc],
    )
    legend = ax.legend(ncol=1)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.grid(which="major", axis="y", visible=True)
    
    # Only plot legend in first plot, remove from all other plots
    if not plot_legend:
        legend.set_visible(False)
    plot_legend = False

    
for ax in axes_iter:
    ax.axis("off")
    
fig.supylabel("Generated electricity [MWh]")

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])